# [OpenCV-Python Tutorial] Classification

In this notebook, we will learn how to perform Object Classification using Visual Bag of Words(VBoW), spatial histogram feature, and SVM classifier.

---

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os, sys
import tarfile
import time
import random
import Pickle as pickle

ModuleNotFoundError: No module named 'Pickle'

In [ ]:
caltech_url = 'http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz'
caltech_filename = '101_ObjectCategories.tar.gz'
caltech_dir = '101_ObjectCategories/'

numTrain = 15
numTest = 15
numClasses = 102
numWords = 600

vocabPath = os.path.join(caltech_dir, 'vocab.pkl')
svmPath = os.path.join(caltech_dir, 'svm_data.dat')

# Dataset

## 1. Download Caltech-101 dataset

In [ ]:
def download_file(url, path):
    import urllib2
    file_name = url.split('/')[-1]
    u = urllib2.urlopen(url)
    f = open(os.path.join(path, file_name), 'wb')
    meta = u.info()
    file_size = int(meta.getheaders("Content-Length")[0])
    print "Downloading: %s Bytes: %s" % (file_name, file_size)

    download_size = 0
    block_size = 8192
    while True:
        buf = u.read(block_size)
        if not buf:
            break
        download_size += len(buf)
        f.write(buf)
        status = "\r%12d  [%3.2f%%]" % (download_size, download_size * 100. / file_size)
        print status,
        sys.stdout.flush()
    f.close()

if not os.path.exists(caltech_dir) or not os.path.exists(os.path.join(caltech_dir, 'airplanes')):
    print 'Downloading Caltech-101'
    if not os.path.exists(caltech_filename) or os.path.getsize(caltech_filename) != 131740031:
        download_file(caltech_url, '.')
    print 'Extracting Caltech-101'
    with tarfile.open(caltech_filename) as tar:
        tar.extractall()

print 'Caltech-101 dataset checked'

## 2. Setup Data

In [ ]:
# Select 15 train/test images for each class
print 'Select 15 train/test images for each class'
classes = os.walk(caltech_dir).next()[1]
train_image_ll = []
test_image_ll = []
for c in classes:
    class_dir = os.path.join(caltech_dir, c)
    ims = [f for f in os.walk(class_dir).next()[2] if f.endswith('.jpg')]
    random.shuffle(ims)
    train_image_ll.append([os.path.join(caltech_dir,c,f) for f in ims[:numTrain]])
    test_image_ll.append([os.path.join(caltech_dir,c,f) for f in ims[numTrain:numTrain+numTest]])
print 'done'

# Show 4 random images
plt.figure(figsize=(16,4))
for i, idx in enumerate(random.sample(range(len(classes)), 4)):
    fpath = random.sample(train_image_ll[idx], 1)[0]
    plt.subplot(1, 4, i+1)
    plt.imshow(cv2.cvtColor(cv2.imread(fpath), cv2.COLOR_BGR2RGB))
    plt.xlabel(fpath)

# Dense SIFT(PHOW) and Visual Words

## 1. SIFT helper functions

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()

# Resize a image if it's too large
def standarizeImage(img):
    rows, cols = img.shape
    if cols > 480:
        img = cv2.resize(img, (480, rows*480/cols))
    return img

# Detect and extract SIFT from a single image
def SIFT(img):
    kp, des = sift.detectAndCompute(img, None)
    return des

# Load an image + Resize if large + Extract SIFT
def SIFT2(img_fpath):
    img = cv2.imread(img_fpath, cv2.IMREAD_GRAYSCALE)
    img = standarizeImage(img)
    kp, des = sift.detectAndCompute(img, None)
    return SIFT(img)

# Dense SIFT(Extract SIFT descriptor in grid points over an image)
def denseSIFT(img, step = 5, size = 7):
    rows, cols = img.shape[:2]
    kp = []
    for x in xrange(step,cols,step):
        for y in xrange(step,rows,step):
            kp.append(cv2.KeyPoint(x, y, size))
    kp, des = sift.compute(img, kp)
    return des

# Load an image + Resize if large + Dense SIFT
def denseSIFT2(img_fpath, step = 10):
    img = cv2.imread(img_fpath, cv2.IMREAD_GRAYSCALE)
    img = standarizeImage(img)
    return denseSIFT(img, step)

## 2. Visual Words(Train Vocabulary)

In [ ]:
# Train vocabulary
if not os.path.exists(vocabPath):
    # Get PHOW features from 30 random training image to build a dictionary
    print "Extracting PHOW features some training images..."
    PHOW_descrs = []
    temp = [item for sublist in train_image_ll for item in sublist]
    temp = random.sample(temp, 30)
    for fpath in temp:
        des = denseSIFT2(fpath)
        PHOW_descrs.append(des)
    PHOW_descrs = np.concatenate(PHOW_descrs, axis=0)
    print "Total %d PHOW features" % PHOW_descrs.shape[0]
    
    # Quantize the descriptors to get the visual words
    print "Running K-means clustering (%d -> %d)..." % (PHOW_descrs.shape[0], numWords)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 500, 1.0)
    attempts = 10
    flags = cv2.KMEANS_RANDOM_CENTERS
    start_time = time.time()
    retval, bestLabels, vocab = cv2.kmeans(PHOW_descrs, numWords, None, criteria, attempts, flags)
    print('Elapsed time: %.6fs' % (time.time() - start_time))
    
    print 'Saving...' 
    with open(vocabPath, 'wb') as fd:
        pickle.dump(vocab, fd)
else:
    print "Load the trained visual words..."
    with open(vocabPath) as fd:
        vocab = pickle.load(fd)

print 'Done!'

In [ ]:
import collections
myCounter = collections.Counter(bestLabels.flatten())
plt.plot(sorted(myCounter.values()))

# Classification - SVM

## Spatial Histogram

In [ ]:
bf = cv2.BFMatcher()

def getImageDescriptor(img, step=5, size=7):
    img = standarizeImage(img)
    cols, rows = img.shape[:2]
    
    # Extracting denseSIFT and BoW
    kp = []
    for x in xrange(step,cols,step):
        for y in xrange(step,rows,step):
            kp.append(cv2.KeyPoint(x, y, size))
    kp, des = sift.compute(img, kp)
    matches = bf.knnMatch(des, vocab, k=1)
    words = [m[0].trainIdx for m in matches]
    
    # Spatial Binning - 2x2
    binX = 2; binY = 2;
    temp = np.zeros((binX, binY, numWords), dtype=np.float32)
    for k, w in zip(kp, words):
        i = (int)(k.pt[0]) * binX / cols
        j = (int)(k.pt[1]) * binY / rows
        temp[i, j, w] += 1
    for i in range(binX):
        for j in range(binY):
            temp[i, j, :] /= np.sum(temp[i, j, :])
    temp = temp.flatten()
    hist = temp
    
    # Spatial Binning - 4x4
    binX = 4; binY = 4;
    temp = np.zeros((binX, binY, numWords), dtype=np.float32)
    for k, w in zip(kp, words):
        i = (int)(k.pt[0]) * binX / cols
        j = (int)(k.pt[1]) * binY / rows
        temp[i, j, w] += 1
    for i in range(binX):
        for j in range(binY):
            temp[i, j, :] /= np.sum(temp[i, j, :])
    temp = temp.flatten()
    hist = np.concatenate((hist, temp), axis=0)
    
    hist /= np.sum(hist)
    return hist    

## 1. Genarate spatial histogram for Training Images

In [ ]:
# Extract spartial histogram for all training images
print 'Extract spartial histogram for all training images'
BoW_train_ll = []
for c, image_list in zip(classes, train_image_ll):
    print c, 
    temp = []
    for fpath in image_list:
        img = cv2.imread(fpath, cv2.IMREAD_GRAYSCALE)
        temp.append(getImageDescriptor(img))
    BoW_train_ll.append(temp)

print '\ndone!'

## 2. Train SVM

In [ ]:
# Setting training data(BoW, label) for SVM
print 'Setting training data(BoW, label) for SVM'
train_bow = []
train_labels = []
numSpartialHist = len(BoW_train_ll[0][0])
for i, BoW_list in enumerate(BoW_train_ll):
    for bow in BoW_list:
        # Hellinger's kernel for each training instance(bow)
        temp = np.sqrt(bow)
        temp -= np.average(temp)
        if np.std(temp) != 0:
            temp /= np.std(temp)
        train_bow.append([temp])
    train_labels.extend([i] * len(BoW_list))
train_bow = np.concatenate(train_bow, axis=0)
train_labels = np.array(train_labels, dtype=np.int).reshape((numTrain*numClasses,1))
# => train_bow: (1530, 12000), train_labels: (1530,1)

print "train_bow: " + str(train_bow.shape)
print "train_labels: " + str(train_labels.shape)

In [ ]:
# Train SVM
print 'Training SVM...'
svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setC(0.01)
svm.setTermCriteria((cv2.TERM_CRITERIA_COUNT, 10, 1.0))

start_time = time.time()
# svm.train(SVM_train_data)
svm.train(train_bow, cv2.ml.ROW_SAMPLE, train_labels)
print('Elapsed time: %.6fs' % (time.time() - start_time))

# print 'Saving SVM...'
# svm.save(svmPath)
print 'Done!'

In [ ]:
train_preds = svm.predict(train_bow)[1]
print('Training Accuracy: %.6f' % np.average(train_preds == train_labels))

## 3. Test trained SVM

In [ ]:
# Extract spartial histogram for all test images
print 'Extract spartial histogram for all test images\n'
BoW_test_ll = []
for c, image_list in zip(classes, test_image_ll):
    print c, 
    temp = []
    for fpath in image_list:
        img = cv2.imread(fpath, cv2.IMREAD_GRAYSCALE)
        temp.append(getImageDescriptor(img))
    BoW_test_ll.append(temp)

print '\ndone!'

In [ ]:
# Setting test data(BoW, label) for SVM
print 'Setting test data(BoW, label) for SVM'
test_bow = []
test_labels = []
numSpartialHist = len(BoW_test_ll[0][0])
for i, BoW_list in enumerate(BoW_test_ll):
    for bow in BoW_list:
        temp = np.sqrt(bow)
        temp -= np.average(temp)
        if np.std(temp) != 0:
            temp /= np.std(temp)
        test_bow.append([temp])
    test_labels.extend([i] * len(BoW_list))
test_bow = np.concatenate(test_bow, axis=0)
test_labels = np.array(test_labels, dtype=np.int).reshape((numTest*numClasses,1))
# => test_bow: (1530, 200), test_labels: (1530,1)

print test_bow.shape
print test_labels.shape

In [ ]:
test_preds = svm.predict(test_bow)[1]
print('Test Accuracy: %.6f' % np.average(test_preds == test_labels))

## 4. Sample Results

In [ ]:
# Show 8 random images
plt.figure(figsize=(16,10))
for i, idx in enumerate(random.sample(range(len(classes)), 8)):
    # Extract Image Descriptor
    fpath = random.sample(test_image_ll[idx], 1)[0]
    img = cv2.imread(fpath, cv2.IMREAD_GRAYSCALE)
    bow = getImageDescriptor(img)
    
    # Hellinger's kernel
    temp = np.sqrt(bow)
    temp -= np.average(temp)
    if np.std(temp) != 0:
        temp /= np.std(temp)
    
    # Prediction
    test_preds = (int)(svm.predict(temp.reshape((1,12000)))[1][0,0])
    
    plt.subplot(2, 4, i+1)
    plt.imshow(cv2.cvtColor(cv2.imread(fpath), cv2.COLOR_BGR2RGB))
    plt.xlabel("GT: %s / Predict: %s" % (classes[idx], classes[test_preds]))

## Practice: Classification for your own image

Choose Classify your own test image using our classification.

- Place your image in the path where this notebook can access
- Load an image as grayscale
- Extract Dense SIFT BoW with `getImageDescriptor()`
- Normalize the bow with Hellinger's kernel
- Predict using the SVM classifier we have trained so far(`svm` instance)

In [ ]:
img_fpath = 'images/dragonfly.jpg'  # Path to your own image
img = cv2.imread(img_fpath, cv2.IMREAD_GRAYSCALE)

#================ YOUR CODE HERE ===================


prediction = 0  # Predicted class of your image
#===================================================

plt.imshow(cv2.cvtColor(cv2.imread(img_fpath), cv2.COLOR_BGR2RGB))
plt.xlabel("Prediction: %s" % classes[prediction])

In [ ]:
# Desired output